In [4]:
%matplotlib inline

from time import time
import numpy as np
import matplotlib.pyplot as plt

import pandas as pd

import os
from os import path 
from time import time

In [5]:
os.environ.setdefault("DJANGO_SETTINGS_MODULE", "TRAS.settings")
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"
import django
from django.core.exceptions import ObjectDoesNotExist
django.setup()
from django.db import transaction
from django.conf import settings

from django_pandas.io import read_frame

from proto.models import Attraction, Review

from nltk import tokenize
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from asgiref.sync import sync_to_async
import nltk

In [6]:
# 디장고 all 리뷰 데이터 불러오고. bulk load?
def sentiment(sentence_list):
    '''
    리뷰를 문장 단위로 쪼개놓은 데이터를 받아서
    각 문장에 대한 감정 분석을 한다.
    '''
    sid = SentimentIntensityAnalyzer()
    
    result_list = []
    for sentence in sentence_list:
        # print(sentence) 샘플만 작게 해서 테스트
        result_list.append((sentence ,sid.polarity_scores(sentence)['compound']))
        '''
        for k in sorted(ss):
            print('{0}: {1}, '.format(k, ss[k]), end='')
        print()
        '''
        #compound: 0.8316, neg: 0.0, neu: 0.254, pos: 0.746,
    return pd.DataFrame(result_list, columns=['sentence','sentiment'])

def sentences_extraction(sent_df, sample_size):
    sent_df.sort_values(by=['sentiment'],inplace=True)
    
    positive_sample = sent_df[-int(sample_size):][::-1]
    negative_sample = sent_df[:int(sample_size)]
    
    return (positive_sample, negative_sample)

#star기반 수축 감정 사전 구축

In [29]:
def attraction_sampling(Attractions_name, All_Reviews_df, start, end):
    result_dict = {}
    for Attraction_name in Attractions_name[start:end]:
        Reviews_df = All_Reviews_df[All_Reviews_df['attraction'] == Attraction_name]
        sentence_list = tokenize.sent_tokenize(" ".join([Review[4] for Review in Reviews_df.values]))
        sent_sent_df = sentiment(sentence_list)
        result_dict[Attraction_name] = sentences_extraction(sent_sent_df, 10)
        print(Attraction_name, " 감정분석 완료!")
    return result_dict        

In [8]:
Attractions_query = Attraction.objects.all()      
Attractions_df = read_frame(Attractions_query)

In [9]:
Attractions_name = Attractions_df.name.tolist()

In [10]:
Attractions_name[:10]

['Gyeongbokgung Palace',
 'Bukhansan National Park',
 'Myeongdong Shopping Street',
 'Changdeokgung Palace',
 'The War Memorial of Korea',
 'National Museum of Korea',
 'Insadong',
 'Trickeye Museum Seoul',
 'Lotte World Tower & Mall',
 'KTX (Korea Train Express)']

In [11]:
query = Review.objects.all()
All_Reviews_df = read_frame(query)

In [26]:
All_Reviews_df[::1000].values[0][4]

'I had the chance to visit the site after a business meeting nearby, this is very impressive and gives an idea about history of Korea'

In [28]:
attraction_review_sample = attraction_sampling(Attractions_name, All_Reviews_df, start=0, end=10)

In [ ]:
import pprint
pp = pprint.PrettyPrinter(indent = 4)

In [44]:
def pprint_review_sample(attraction_review_sample, i):
    key = list(attraction_review_sample.keys())[i]
    (pos_df, neg_df) = attraction_review_sample[key]
    print(key)
    for pos_sent in pos_df.values:
        print(pos_sent[1])
        print(pos_sent[0].split('.')[0])
    for neg_sent in neg_df.values:
        print(neg_sent[1])
        print(neg_sent[0].split('.')[0])

In [55]:
Attractions_name[:10]

['Gyeongbokgung Palace',
 'Bukhansan National Park',
 'Myeongdong Shopping Street',
 'Changdeokgung Palace',
 'The War Memorial of Korea',
 'National Museum of Korea',
 'Insadong',
 'Trickeye Museum Seoul',
 'Lotte World Tower & Mall',
 'KTX (Korea Train Express)']

In [ ]:
# 아래는 리뷰가 다 사라졌길래 다시 bulk create한 내용

In [54]:
def review_date(date_string):
    dict = {
        ' January ':'-01-01',
        ' February ':'-02-01',
        ' March ':'-03-01',
        ' April ':'-04-01',
        ' May ':'-05-01',
        ' June ':'-06-01',
        ' July ':'-07-01',
        ' August ':'-08-01',
        ' September ':'-09-01',
        ' October ':'-10-01',
        ' November ':'-11-01',
        ' December ':'-12-01',
    }
    
    return date_string[-4:] + dict[date_string[:-4]]

In [66]:
review_path = 'C:/Users/taehee/Documents/GitHub/tripReviewAnalysisSystem/크롤러-전처리/원시자료/'
Attraction_list = Attraction.objects.all()
Reviews_list = []
for the_Attraction in Attraction_list:
    Attraction_Reviews_df = pd.read_csv(review_path+ the_Attraction.name+".csv", engine='python', index_col = 0, encoding = 'utf-8')
    for Review_list in Attraction_Reviews_df.values[2:]:
        Reviews_list.append(Review(attraction = the_Attraction,
                       star = Review_list[0],
                      title = Review_list[1],
                      text = Review_list[2],
                      date = review_date(Review_list[3])))

In [69]:
len(Reviews_list)

18534

In [72]:
for num in range(19):
    start = (num-1)*1000
    if start <0:
        start = 0
    end = num*1000
    if end > 18534:
        end = 18534
    Review.objects.bulk_create(Reviews_list[start:end])

In [61]:
max_len = 0
max_text = ""
for Review in Reviews_list:
    if len(Review.text) > max_len:
        max_text = Review.text
        max_len = len(Review.text)

In [62]:
print(max_len)

9136
